# Refined Power Analysis for RE

This notebook explores using the RPA attack for reverse-engineering.

In [ ]:
from binascii import hexlify
from copy import copy

from pyecsca.ec.params import get_params
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.mod import Mod

### TODO
 - Multipliers differ in their "intermediate scalars" that they compute while computing a scalarmult.
     - How to explore this?
     - LTR vs RTL? prefixes vs suffixes vs windows?
 - In the RPA attack, which attacks static ECDH, one is able to detect a particular multiple of the input point is computed, by crafting a special input point and using a power side-channel.
     - How to explore this?
     - It is an "oracle", you are able to ask questions like "is $[k]P$ computed in a given implementation when it is doing scalarmult of $[r]P$?"
     - Show the RPA attack. Existence of $P_0$ point, then, when you do $[k^{-1}]P_0 = P$ you get a point such that $[k]P = [k][k^{-1}]P_0 = P_0$. Then get two groups of traces of the implementation computing scalarmult with a secret (fixed) scalar, one group with random input points, other with point $P$ for some $k$. If $[k]P$ is computed, you will see a difference between these two groups at some point, because of the zero coordinate. To spot the difference, basically do DPA: average the groups and subtract their average traces, then see spike (peak detect or threshold).
 - Putting these two things together, we see that we can build a reverse-engineering method out of them.

## Scalar multipliers
Scalar multipliers come in all sorts of different shapes, sizes and colors. 🌈

In [ ]:
from pyecsca.ec.mult import LTRMultiplier, RTLMultiplier
from pyecsca.sca.re.rpa import MultipleContext

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Compare the chain of multiples that is computed in the left-to-right vs right-to-left scalar multiplier for a given fixed scalar.
- What do these intermediate multiples (scalars) represent in the two cases?
- When are the chains the same? When are the sets of multiples the same?

**Docs**<br/>
[LTRMultiplier](https://neuromancer.sk/pyecsca/api/pyecsca.ec.mult.binary.html#pyecsca.ec.mult.binary.LTRMultiplier)<br/>
[RTLMultiplier](https://neuromancer.sk/pyecsca/api/pyecsca.ec.mult.binary.html#pyecsca.ec.mult.binary.RTLMultiplier)<br/>
[MultipleContext](https://neuromancer.sk/pyecsca/api/pyecsca.sca.re.rpa.html#pyecsca.sca.re.rpa.MultipleContext)

In [ ]:
params = get_params("secg", "secp256r1", "projective")
add = params.curve.coordinate_model.formulas["add-2007-bl"] # The formulas do not matter, but we need some
dbl = params.curve.coordinate_model.formulas["dbl-2007-bl"] # The formulas do not matter, but we need some

ltr_mult = LTRMultiplier(add, dbl)
rtl_mult = RTLMultiplier(add, dbl)

# Task: Fix a scalar
k = ...
P = params.generator

with local(MultipleContext()) as ltr_ctx:
    # Task: Init and multiply using the LTR multiplier here

with local(MultipleContext()) as rtl_ctx:
    # Task: Init and multiply using the RTL multiplier here 

# Task: Compare the ltr_ctx and rtl_ctx.
# You can look at their .points and .parents attributes.

## RPA attack

In [11]:
from pyecsca.sca.re.rpa import rpa_point_0y, rpa_point_x0, rpa_input_point

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

**Docs**<br/>
[rpa_point_0y](https://neuromancer.sk/pyecsca/api/pyecsca.sca.re.rpa.html#pyecsca.sca.re.rpa.rpa_point_0y)<br/>
[affine_multiply](https://neuromancer.sk/pyecsca/api/pyecsca.ec.curve.html#pyecsca.ec.curve.EllipticCurve.affine_multiply)

In [16]:
P0 = rpa_point_0y(params)

# Task: Create a point P, such that P0 occurs in the scalar multiplication of [k]P using the LTR multiplier
#       but not the RTL multiplier.
# Hint: You can use params.curve.affine_multiply.
# Hint: The rpa_input_point method does this all, but do not use it.

## Putting it together